**Import Libraries**

In [2]:
#!pip install plotly
import plotly.express as px
from sklearn.preprocessing import StandardScaler, PolynomialFeatures, MinMaxScaler
from sklearn.decomposition import PCA
import requests
from bs4 import BeautifulSoup
import re
import os
import matplotlib as mpl
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
import glob
import seaborn as sns
from scipy import stats
from sklearn.linear_model import Lasso, LassoCV
from sklearn.linear_model import LinearRegression
from sklearn.metrics import mean_squared_error
from sklearn.model_selection import cross_validate
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import PolynomialFeatures
from sklearn.utils import shuffle
from warnings import simplefilter
simplefilter('ignore', category=FutureWarning)

Please note, we will be collaborating via Git. Please find our project at https://github.com/ppich1169/fertilityProjectCs109

# Milestone 1: Proposal

Over the past fifty years, fertility rates in the US have plummeted and are currently at a historic low. Conversations about why fertility has fallen so substantially and how we can address the implications of this shift for government programs like social security have been quite salient in recent public discourse and in the 2024 election cycle. 

Interestingly, there is significant variation in fertility rates across US states. We’d like to understand the relative importance of various factors in determining a state’s fertility rate. 

We plan to run a multiple regression of fertility rate (can get state-by-state here from the CDC’s National Center for Health Statistics) on a number of regressors

**Goal:** Create a regression that can predict the fertility rate of a state. Then, analyze coefficients and/or use causal inference to understand why fertility rate is going down

# Milestone 2: Preprocessing

## 1. Access the data that you will be using for the final project by downloading, collecting, or scraping* from the relevant source(s)

### Response Variable

Our response variable is **fertility rate by state over time** which can be found at https://www.cdc.gov/nchs/pressroom/sosmap/fertility_rate/fertility_rates.htm. 

We accessed it via download and saved it as `fertility_rate_census.csv`. 

Please note, fertility rate is defined as  **total number of births per 1,000 women aged 15-44** and our dataset looks at fertility rate for each of the 50 states over 9 years (2014-2022).

### Predictors
_We used our previous knowledge and assumptions to create an **X** dataset of predictors that we believe may influence fertility rates_

Because fertility rate is evaluated statewide , and states vary significantly in population size, we have decided that for all of the predictors, we are going to essentially **normalize** them by looking at the percentage of each state that fall into a specific category. 

**CENSUS DATA**

We collected essential demographic data from the Census Bureau  to analyze important trends across the U.S. population. Key indicators included race, socioeconomic status (using the Supplemental Poverty Measure, or SPM, to capture household income and the percentage of households below the poverty line), education level (representing different shares of educational attainment), and immigration status (percentage of foreign-born individuals). We also included age, sex, year, and state to enable aggregation across different regions and demographics for a more comprehensive analysis.

We accessed this data by downloading it from the website, cleaned it, and and organized it into several data frames—foreignborn_df, SPM_df, race_df, and education_df—within our data folder, facilitating clear segmentation of data sources for efficient management and analysis. For socioeconomic status, we utilized the ACS SPM (Supplemental Poverty Measure) State tables from the U.S. Census Bureau, which offered a broader measure of poverty than the traditional poverty threshold. The SPM accounts for not only cash income but also non-cash benefits, such as food assistance, as well as necessary expenses like taxes and healthcare costs. This data allowed us to estimate the percentage of households below the poverty line, providing a critical indicator of financial need at the state level.

To assess immigration, we utilized the percentage of foreign-born populations by state for 2022, sourced from Statista and aggregated from Census Bureau data. This data, while informative, is limited by its aggregated format, representing only documented immigrants and potentially underrepresenting undocumented populations. Our education data, also from the Census Bureau, included different shares of educational attainment across age groups, rather than focusing solely on high school graduates. We calculated weighted percentages of each educational level by state to accurately reflect the distribution of education within different populations, adjusting these percentages by each state’s total population to ensure comparability. Our race data, sourced from the Census Bureau, maintained consistency across data frames for accuracy and comparability.

In processing and cleaning the data, we standardized variables such as age, sex, year, and state across all data frames, enabling seamless aggregation. Additionally, it is important to mention there are potential discrepancies, particularly the immigration and education data due to inherent limitations in coverage. 

In [4]:
# Define path pattern to load all "education.csv" files in the 'data' folder
file_pattern = "data/*education.csv"
all_files = glob.glob(file_pattern)

# Initialize empty list to collect data frames
df_list = []

# Process each education file
for file in all_files:
    # Extract year from the file name (assuming the format 'YYYYeducation.csv')
    year = file.split('/')[-1][:4]

    # Load the CSV file
    df = pd.read_csv(file)

    # Drop "Population 25 years and over" and the nine rows following it
    pop_25_index = df[df.iloc[:, 0] == "Population 25 years and over"].index
    if not pop_25_index.empty:
        df = df.drop(index=range(pop_25_index[0], pop_25_index[0] + 10)).reset_index(drop=True)

    # Identify the first column name dynamically (the label column)
    label_column = df.columns[0]

    # Keep only the first column (label) and columns ending with "!!Percent Females!!Estimate"
    columns_to_keep = [label_column] + [col for col in df.columns if col.endswith("!!Percent Females!!Estimate")]
    df = df[columns_to_keep]

    # List to store processed data
    selected_data = []

    # Process each age group section to calculate weighted averages
    current_population = None
    total_population = 0  # To accumulate total population for the age group
    for index, row in df.iterrows():
        label = row[label_column]

        # Check if row represents a population group (e.g., "Population 18 to 24 years")
        if "Population" in label:
            # Extract population number from label and set as current population group
            try:
                total_population = int(label.split()[1]) if label.split()[1].isdigit() else None
            except ValueError:
                total_population = None
            current_population = total_population  # Set total for this age group
        elif current_population and "Percent" not in label:
            # Calculate weighted value for educational attainment percentages based on the total population
            for col in df.columns[1:]:  # Exclude the label column
                # Extract the state name from the column title
                state = col.split("!!")[0]
                
                # Get the percentage as a string and convert to a numeric type
                percent_str = row[col]

                try:
                    # Attempt to convert percent_str to a float, after stripping % and whitespace
                    percent = float(str(percent_str).replace("%", "").strip()) if percent_str else None
                except ValueError:
                    percent = None  # Set as None if conversion fails

                # Perform calculation only if percent is not None
                if percent is not None:
                    scaled_value = (percent / 100) * total_population
                else:
                    scaled_value = None

                # Append the result to selected_data
                selected_data.append({
                    "Label": label,
                    "Year": year,
                    "State": state,
                    "Female Estimate": scaled_value,
                    "Population": total_population
                })

    # Convert selected_data list into a DataFrame and append to df_list
    temp_df = pd.DataFrame(selected_data)
    df_list.append(temp_df)

# Combine all data frames into a single data frame
education_df = pd.concat(df_list, ignore_index=True)

# Drop rows where "Female Estimate" is NaN
education_df.dropna(subset=["Female Estimate"], inplace=True)

# Display the cleaned DataFrame
education_df


,Label,Year,State,Female Estimate,Population
0,Less than high school graduate,2015,Alabama,2.340,18
1,Less than high school graduate,2015,Alaska,2.430,18
2,Less than high school graduate,2015,Arizona,2.628,18
3,Less than high school graduate,2015,Arkansas,2.070,18
4,Less than high school graduate,2015,California,1.836,18
...,...,...,...,...,...
4987,Bachelor's degree or higher,2016,Washington,3.055,65
4988,Bachelor's degree or higher,2016,West Virginia,2.990,65
4989,Bachelor's degree or higher,2016,Wisconsin,2.405,65
4990,Bachelor's degree or higher,2016,Wyoming,2.860,65


In [ ]:
# Define path pattern to load all "race.csv" files with a year prefix in the 'data' folder
file_pattern = "data/*race.csv"
all_files = glob.glob(file_pattern)

# Initialize an empty list to collect data frames
df_list = []

# Loop through all matching files
for file_path in all_files:
    # Extract the year from the file name (assuming it is the first part of the file name)
    year = file_path.split('/')[-1].split('race')[0]
    
    try:
        # Load the CSV file, skipping the first two lines and using comma as the delimiter
        df = pd.read_csv(file_path, skiprows=2, delimiter=',')
        
        # Add the year column
        df['Year'] = year
        
        # Drop columns with specific names if they exist
        columns_to_drop = ['American Indian or Alaska Native', 'Native Hawaiian or Pacific Islander', 'Total', 'Footnotes']
        df = df.loc[:, ~df.columns.str.contains('|'.join(columns_to_drop), na=False)]
        
        # Remove rows with "United States" in the first column
        df = df[~df.iloc[:, 0].str.strip().str.lower().eq("united states")]
        
        # Remove rows where "White" is NaN
        df = df.dropna(subset=["White"])
        
        # Append the cleaned dataframe to the list
        df_list.append(df)
    
    except pd.errors.ParserError:
        print(f"Could not parse {file_path}. Skipping this file.")
    except Exception as e:
        print(f"An error occurred with file {file_path}: {e}")

# Concatenate all dataframes in the list
race_df = pd.concat(df_list, ignore_index=True)

# Display the merged dataframe
race_df


In [ ]:
# Load the CSV file and examine its structure
file_path = 'data/SPM.csv'

# Reload the CSV file as a single column and then split based on the tab delimiter
SPM_df = pd.read_csv(file_path)

SPM_df

In [ ]:
# Load the CSV file and examine its structure
file_path = 'data/foreignborn2022.csv'

# Reload the CSV file as a single column and then split based on the tab delimiter
df = pd.read_csv(file_path, header=None)

# Split the single column by tab to separate into "State" and "Percent"
df[['State', 'Percent']] = df[0].str.split('\t', expand=True)

# Drop the original combined column
foreignborn_df = df[['State', 'Percent']]

# Display the first few rows to confirm the split
foreignborn_df


We also care about religion, political makeup, and whether certain abortion laws are in place (all of which are not in the census). We will find them different ways.

**RELIGION DATA**

We decided to determine people's **religiousness** based off of a state's adherence rate (number of people who adhere to their religion across 1000 ) which can be found at (https://www.thearda.com/us-religion/maps/us-state-maps)

We accessed it by webscraping and saved it in `religion_data.csv` in our data folder

In [ ]:
url2020 = "https://www.thearda.com/us-religion/maps/us-state-maps"
url2010 = "https://www.thearda.com/us-religion/maps/us-state-maps?color=orange&m1=2_2_9999_2010"
url2000 = "https://www.thearda.com/us-religion/maps/us-state-maps?color=orange&m1=2_2_9999_2000"
response = requests.get(url2000)
soup = BeautifulSoup(response.content, 'html.parser')

if response.status_code == 200:
    soup = BeautifulSoup(response.content, 'html.parser')
else:
    print(f"Failed to retrieve the webpage. Status code: {response.status_code}")
    
script_tag = soup.find('script', text=re.compile(r'usa_map_div299992000_data = '))
script_content = script_tag.string
start_index = script_content.find('usa_map_div299992000_data =')
semicolon_index = script_content.find(';', start_index)
mapData = script_content[start_index:semicolon_index]

quoted_strings = re.findall(r'"(.*?)"', mapData)
values_strings = re.findall(r'(\d+\.\d+)', mapData)
year = []
for i in range(len(values_strings)):
    year.append(2000)

last_two_chars = [s[-2:] for s in quoted_strings]
print(last_two_chars)
print(len(values_strings))

file_paths = [
    'data/AllReligionAdherence_2000.csv',
    'data/AllReligionAdherence_2010.csv',
    'data/AllReligionAdherence_2020.csv'
]

dfs = []

for file_path in file_paths:
    file_name = os.path.basename(file_path)
    year = file_name.split('_')[-1].split('.')[0]
    df = pd.read_csv(file_path)
    df['Year'] = year
    dfs.append(df)

combined_df = pd.concat(dfs, ignore_index=True)

['UT', 'ND', 'DC', 'SD', 'MA', 'RI', 'MN', 'OK', 'WI', 'NY', 'NE', 'LA', 'IA', 'NM', 'PA', 'CT', 'NJ', 'AR', 'TX', 'IL', 'AL', 'MS', 'KY', 'MO', 'TN', 'KS', 'ID', 'NH', 'SC', 'WY', 'CA', 'NC', 'OH', 'GA', 'MT', 'MD', 'IN', 'MI', 'VA', 'FL', 'DE', 'AZ', 'CO', 'VT', 'ME', 'HI', 'WV', 'AK', 'NV', 'WA', 'OR']
51


/var/folders/bv/cb2mryw56_78gj2rs1gj66c80000gn/T/ipykernel_10416/1691597460.py:2: DeprecationWarning: The 'text' argument to find()-type methods is deprecated. Use 'string' instead.
  script_tag = soup.find('script', text=re.compile(r'usa_map_div299992000_data = '))


**POLITICS DATA**

We decided to determine people's political orienation based off of the Cook Partisan Voting Index (Cook PVI), which is a measure of each state's political leaning relative to the nation as a whole.

The primary challenge with using this index is that the methodology was switched in 2022 to weigh the last presidential election 0.75 and the second to last 0.25, as opposed to the even (50/50) weighting of both years that was used in prior years. We will either figure out how to reweight the outcomes based on the raw data if we can get it or exclude 2022 from our analysis.

The calculation of the index is described in more detail here: https://www.cookpolitical.com/cook-pvi

**Missingness and imbalance**: None in the dataset 

**Scaling:** We will follow a similar methodology to that used in "State-level Political Partisanship Strongly Correlates with Health Outcomes for US Children," (full citation below).* We converted the PVI's to numerical values, with negative values representing Democratic PVIs and positive numbers representing Republican ones (an arbitrary choice). Then, we scaled those values with sklearn's MinMaxScaler() so that states have a rating between 0 and 1 representing how conservative they are, with 1 being most conservatve and 0 being most liberal.

We accessed the data for 2022 from this source: https://datawrapper.dwcdn.net/0djXs/2/ and saved it in cook_pvi_2022.csv in our data folder. We tried scraping the website but the data is not in the html but instead pulled from a source so we were unable to access the data using the same strategy as HW 1.

The 2022 data is saved in `data/cook_pvi_2022.csv.`

*Paul, M., Zhang, R., Liu, B. et al. State-level political partisanship strongly correlates with health outcomes for US children. Eur J Pediatr 181, 273–280 (2022). https://doi.org/10.1007/s00431-021-04203-y

**ABORTION DATA**

We decided to determine state's **abortion laws** based off of how late into pregnancy, abortion is legally allowed which can be found at https://lawatlas.org/datasets/abortion-bans. We chose this dataset because it is the only one on the internet showing abortion bans in the 2014-2022 time frame and how they change (most just show abortion bans now)

We accessed it by downloading it, converting from xlsx to csv, and saved it in `abortion_data.csv` in our data folder

The main thing to consider is that **just because abortion is legal, doesn't mean it is accessible**. Many states may technically allow abortion but only have one clinic, so its not attainable. That said, we have chosen this metric (when is abortion legal), to coincide with current political debate about whether abortion should be legalized. 

There will be significant preprocessing required as the data is in format `Effective Date`, `Valid Through Date` for each law which must simply be converted into year (whichever law was the majority of the year), and each ban `6 weeks`, `8 weeks` etc is categorical! It would make more sense to simply make a variable listing the latest week aborition is legal (0,6,8,12,52 etc).  

## 2. Load the data into a Jupyter notebook and understand the data by examining, among other characteristics of interest, data missingness, imbalance, and scaling issues.

Some issues we preliminary have considered before even inspecting the data includes:


- **under reporting immigration status**: via google, people tend to underreport whether they are immigrants. This is potentially a missingness issue

-  **multicollinearity**: There is most likely a relationship between racial background / household income and an individual's birth country (immigration status) so we can't use both as predictors in the same equation. We don't know how strong this correlation will be so are not worried yet, but would love to talk about it with a TF

- **is this just too much data??**: looking at each of these attributes for each state for each year may just be too many dimensions. Is there really a big difference accross years? Should we just look at 1? If so, which? 

Now we are going to import and inspect each dataset, looking for missingness, imbalance, and scaling issues!

### Fertility Rate Data

In [ ]:
fertility_rate = pd.read_csv('data/fertility_rate_census.csv')
print(fertility_rate.isna().sum(axis=0))
fertility_rate.describe()

YEAR              0
STATE             0
FERTILITY RATE    0
BIRTHS            0
URL               0
dtype: int64


,YEAR,FERTILITY RATE,BIRTHS
count,451.000000,451.000000,451.000000
mean,2018.008869,60.057871,75783.962306
std,2.588850,6.420758,86837.134690
min,2014.000000,44.300000,5133.000000
25%,2016.000000,56.000000,21758.500000
50%,2018.000000,60.200000,55971.000000
75%,2020.000000,63.500000,86486.000000
max,2022.000000,80.000000,502879.000000


In [ ]:
pivoted_fertility = fertility_rate.pivot(index='STATE', columns='YEAR', values='FERTILITY RATE')
pivoted_fertility[pivoted_fertility.isna().any(axis=1)]

YEAR,2014,2015,2016,2017,2018,2019,2020,2021,2022
STATE,,,,,,,,,
District of Columbia,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,57.3


Things to Consider:

**Missingness**: While there is no empty cell in the original dataset, we see if we pivot it by year, Washington DC only shows up in 2022. We can simply delete this row as Washington DC isn't technically a state. This is Missing at Random because we know the reason there was no Washington DC from 2014-2021 is that DC isn't considered a state

**Imbalance**: Since we aren't looking at different classes there is no imbalance

**Scaling**: By considering Fertility Rate (and not Number of Births), we are essentially normalizing our data as we are dividing it by total population. This will be enough in order to scale the data as it takes into account the populations of each state such that no state is overly weighed. In addition, since fertility rate is a response variable, not a predictor, we don't have to consider how large our data is in relation to other variables. Thus, we don't have to scale it any further.

**Other**: We need to encode the state variable as categorical if it will be used in modeling by creating dummy variables or one-hot encoding. This issue will be true for all datasets!

### Census Data

In [ ]:
"""
census_data = pd.read_csv('data/')
print(census_data.isna().sum(axis=0))
census_data.describe()
"""

"\ncensus_data = pd.read_csv('data/')\nprint(census_data.isna().sum(axis=0))\ncensus_data.describe()\n"

Things to Consider:

**Missingness**:

**Imbalance**:

**Scaling**:

**Other**:

### Religion Data

In [ ]:
religion_data = pd.read_csv('data/religion_data.csv')
print(religion_data.isna().sum(axis=0))
religion_data.describe()

State                      0
Adherence Rate per 1000    0
Year                       0
dtype: int64


,Adherence Rate per 1000,Year
count,153.000000,153.00000
mean,489.381569,2010.00000
std,101.640693,8.19178
min,272.190000,2000.00000
25%,414.860000,2000.00000
50%,492.770000,2010.00000
75%,555.040000,2020.00000
max,791.060000,2020.00000


Things to Consider:

**Missingness**: After webscraping ARDA, there are no missing values in any cells, but definitely in the number of years (153 vs 451 above). This is because this index is only calculated every 10 years (2000, 2010, 2020). We have to decide how we want to impute this data. One option is to create a line going through the 2000, 2010, and 2020 datapoints, and then impute the data that would sit on this line for 2011-2019 and 2021-2022. 

**Imbalance**: Although the actual number of state datapoints are equal and fine, the source of the imbalance stems from which religions do these data sets stem from. All the main categories listed on the Association of Religious Digital Archives are branches of Christianity. I have no idea if these are actually the most popular religions in the US or is the source biased towards it.

**Scaling**: Since adherence rates range widely (e.g., from 272.19 to 791.06), standardizing or normalizing values might be beneficial, particularly if this data will be used for machine learning or statistical modeling. Standardization (e.g., Z-score) could be applied if you want each adherence rate centered around zero, while min-max normalization scales values between a range like [0, 1].

### Politics Data

In [3]:
pol_df = pd.read_csv('data/cook_pvi_2022.csv')

### Abortion Data

In [ ]:
abortion_data = pd.read_csv('data/abortion_data.csv')

In [ ]:
columns = ["State","Effective Date","Valid Through Date","Bans_gest_4 weeks postfertilization (6 weeks LMP) " ,"Bans_gest_6 weeks postfertilization (8 weeks LMP) " ,"Bans_gest_8 weeks postfertilization (10 weeks LMP)","Bans_gest_10 weeks postfertilization (12 weeks LMP) " ,"Bans_gest_12 weeks postfertilization (14 weeks LMP)","Bans_gest_13 weeks postfertilization (15 weeks LMP)","Bans_gest_16 weeks postfertilization (18 weeks LMP) ","Bans_gest_18 weeks postfertilization (20 weeks LMP)","Bans_gest_19 weeks postfertilization (21 weeks LMP)","Bans_gest20 weeks postfertilization (22 weeks LMP)","Bans_gest_21 weeks postfertilization (23 weeks LMP) " ,"Bans_gest_22 weeks postfertilization (24 weeks LMP)","Bans_gest_24 weeks postfertilization (26 weeks LMP)","Bans_gestViability","Bans_gest_Fetus is capable of feeling pain","Bans_gest_3rd trimester"]
abortion_data = abortion_data[columns]
new_names = {
    "State": "state",
    "Effective Date": "start",
    "Valid Through Date": "end",
    "Bans_gest_4 weeks postfertilization (6 weeks LMP) ": "4",
    "Bans_gest_6 weeks postfertilization (8 weeks LMP) ": "6",
    "Bans_gest_8 weeks postfertilization (10 weeks LMP)": "8",
    "Bans_gest_10 weeks postfertilization (12 weeks LMP) ": "10",
    "Bans_gest_12 weeks postfertilization (14 weeks LMP)": "12",
    "Bans_gest_13 weeks postfertilization (15 weeks LMP)": "13",
    "Bans_gest_16 weeks postfertilization (18 weeks LMP) ": "16",
    "Bans_gest_18 weeks postfertilization (20 weeks LMP)": "18",
    "Bans_gest_19 weeks postfertilization (21 weeks LMP)": "19",
    "Bans_gest20 weeks postfertilization (22 weeks LMP)": "20",
    "Bans_gest_21 weeks postfertilization (23 weeks LMP) ": "21",
    "Bans_gest_22 weeks postfertilization (24 weeks LMP)": "22",
    "Bans_gest_24 weeks postfertilization (26 weeks LMP)": "24",
    "Bans_gestViability": "24", #chose via google
    "Bans_gest_Fetus is capable of feeling pain": "25", #chose via google
    "Bans_gest_3rd trimester": "28" #chose via google
}
abortion_data = abortion_data.rename(columns=new_names)

abortion_processed = abortion_data[['state', 'start', 'end']].copy()
abortion_processed['latest_abortion'] = abortion_data[abortion_data.columns].apply(
    lambda row: next((int(col) for col in abortion_data.columns  if str(row[col]) == "1"), 40), axis=1)

abortion_processed['start'] = pd.to_datetime(abortion_processed['start'])
abortion_processed['end'] = pd.to_datetime(abortion_processed['end'])
abortion_processed['year'] = abortion_processed['start'].dt.year
abortion_processed['length'] = (abortion_processed['end'] - abortion_processed['start']).dt.days
abortion_processed = abortion_processed.loc[abortion_processed.groupby(['state', 'year'])['length'].idxmax()]
abortion_processed = abortion_processed.drop(columns=['start', 'end', 'length'])

abortion_processed.head()

,state,latest_abortion,year
0,Alabama,20,2018
2,Alabama,20,2019
3,Alabama,20,2022
4,Alaska,40,2018
5,Arizona,18,2018


In [ ]:
print(abortion_processed.isna().sum(axis=0))
print("missing from entire dataframe", 50*5-len(abortion_processed))
abortion_processed.describe()

state              0
latest_abortion    0
year               0
dtype: int64
missing from entire dataframe 113


,latest_abortion,year
count,137.000000,137.000000
mean,25.598540,2019.729927
std,10.328758,1.647206
min,4.000000,2018.000000
25%,20.000000,2018.000000
50%,20.000000,2019.000000
75%,40.000000,2021.000000
max,40.000000,2022.000000


Things to Consider:

**Missingness**: We see while there are no null cells in our dataset, there are 113 "missing cells" from what would be expected (one for every state for every year). This is simply because of how our dataset was made -- there is only a new row if a new law is enacted (not one per year). Thus, if we are missing a row for a certain year, we can simply **impute** the data by copying the previous row. 

**Imbalance**: There is no imbalance as there is no population sampling. 

**Scaling**: We don't need to scale the data by itself but might scale data as a whole later so they don't have stds that are too high. 

**Other**: We decided to change our data from a 1-hot encoded dataset, to 1 quantitative varaible. We did this because there seems to be a clear relatonship between the previously encoded categories (a 10 week ban is stronger than a 15 week ban and less strong than a 4 week ban). Thus, we made it ordinal. In additon, note that if there was no abortion ban, we arbitrarily set this ordinal variable to 40, the length of the average pregancy. It might make some sense to add an **indicator variable** to be 0 if there is no abortion ban at all (40 weeks) and 1 if there is an abortion ban. 

## 3. Understand and describe the preprocessing required such that data is in a form amenable to later downstream tasks such as visualizing and modeling, as is appropriate to the specific project goals.



One of our biggest considerations is that our predictor variables, X, is essentially 3d (reshaped), not 2d. We are looking at a bunch of predictors **over state** and **over time**. This might make our predictions complicated, especially because we are trying to see **why** fertility is changing over time (so the reason that there are less babies in 2022 can't simply be that it is 2022). Thus, it might make more sense to choose only **one year** to regress on. Also, it may be hard to **visualize** multiple years and states simultaniously as, for example, if we drew a map and colored each state by predictor category, we would have to choose one specific time to do so (and vice versa).  PLEASE LET US KNOW WHAT YOU THINK!

It also may make sense to just try to minimize predictors in general as we don't want to get too specific!

### This is how we envision generally preprocessing...

******DO THIS CELL TUESDAY, 29th!

### One way to preliminarily understand why fertility rate is decreasing is to look at where it changes over time, and think about what factors dominate those areas.... aka do PCA

In [ ]:
pivoted_fertility = fertility_rate.pivot(index='STATE', columns='YEAR', values='FERTILITY RATE')
X_std =  StandardScaler().fit_transform(pivoted_fertility.fillna(0)) 
pca = PCA(n_components=1) 
principal_components = pca.fit_transform(X_std)

states = pivoted_fertility.index
pca_states = pd.DataFrame(data=principal_components, columns=['PC1'], index=states).reset_index()
pca_states.columns = ['state', 'PC1']

print("this accounts for", np.round(100*pca.explained_variance_ratio_[0], 2), "% variance")

this accounts for 90.25 % variance


In [ ]:
fig = px.choropleth(
    pca_states,
    locations='state',
    locationmode="USA-states",
    color='PC1',
    scope="usa",  
    range_color=[-5,5],
    color_continuous_scale=['purple', 'white', 'green'], 
    labels={'PC1': 'Principal Component Value'}
)

fig.update_layout(title_text="What states have the highest variation in fertility rates?")
fig.show()


We see that the first principle componetent represents 90% of varition over state and time. Thus, most of the change (net decrease) in fertility rate is seen when the coasts (purples) move strongly in one direction (presumably decrease given the net decrease) and the center of the US moves slightly in the other. We can use this information to understand what variables would be good predictors, aka the ones that match the coloring above. For example, a wealth distribution or political map would see similar differences with one color on the coast and another in the center. Thus we know that looking at household income and political affiliation are going to be really good predictor variables!!

Below is the preprocessing for political data. We scale the Cook PVI scores to be on a scale of 0 to 1, where 1 is most conservative and 0 is least conservative, as described above, as well.

In [5]:
#Drop DC because it's not a state: 
pol_df = pol_df[pol_df['State'] != 'District of Columbia']
pol_df.reset_index(drop=True, inplace=True)

unscaled_ratings=[]
for i in range(len(pol_df)):
    magnitude = re.search(r"(?<=\+).*", pol_df['2022_PVI'][i])
    magnitude = int(magnitude.group())
    
    if pol_df['2022_PVI'][i][0] == 'R':
        unscaled_ratings.append(magnitude)
    else:
        unscaled_ratings.append(magnitude * -1)

pol_df['unscaled_rating'] = unscaled_ratings

pol_rating_scaler = MinMaxScaler()
scaled_pol_ratings = pol_rating_scaler.fit_transform(pol_df['unscaled_rating'].values.reshape(-1, 1))

pol_df['scaled_rating'] = scaled_pol_ratings
pol_df.head()

pol_df['unscaled_rating'].min()

pol_df

,State,2022_PVI,2020_Biden,2020_Trump,2016_Clinton,2016_Trump,unscaled_rating,scaled_rating
0,Alabama,R+15,36.60%,62.00%,34.40%,62.10%,15,0.756098
1,Alaska,R+8,42.80%,52.80%,36.60%,51.30%,8,0.585366
2,Arizona,R+2,49.40%,49.10%,44.60%,48.10%,2,0.439024
3,Arkansas,R+16,34.80%,62.40%,33.70%,60.60%,16,0.780488
4,California,D+13,63.50%,34.30%,61.70%,31.60%,-13,0.073171
5,Colorado,D+4,55.40%,41.90%,48.20%,43.30%,-4,0.292683
6,Connecticut,D+7,59.30%,39.20%,54.60%,40.90%,-7,0.219512
7,Delaware,D+7,58.70%,39.80%,53.10%,41.70%,-7,0.219512
8,Florida,R+3,47.90%,51.20%,47.80%,49.00%,3,0.463415
9,Georgia,R+3,49.50%,49.20%,45.60%,50.80%,3,0.463415
